<a href="https://colab.research.google.com/github/BagusDAriAwan/creditcard-fraud/blob/master/IForest_ULB_hasil_pred_normal_supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import math
import random as rn
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb
sb.set_style(style="whitegrid")
sb.set_color_codes()
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.decomposition import PCA
from sklearn.externals import joblib
import time

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def confussion_matrik(actual,predict):
    TP,FP,FN,TN = 0,0,0,0
    for i,val in enumerate(actual):
        if val == 0:
            if val == predict[i]:
                TN += 1
            else:
                FP += 1
        if val == 1:
            if val == predict[i]:
                TP += 1
            else:
                FN += 1
    return TP,FP,FN,TN
 
def acc_sens_spec(actual,predict):
    TP,FP,FN,TN = confussion_matrik(actual,predict)
# akurasi
    if (TP+FP+FN+TN) == 0 :
        accuracy = 0 
    else :
        accuracy = (TP+TN)/(TP+FP+FN+TN)
        
# sensitivity
    if (TP+FN) == 0 :
        sensitivity = 0
    else :
        sensitivity = TP/(TP+FN)
        
# specifity    
    if (TN +FP) == 0 :
        specifity = 0
    else :
        specifity = TN/(TN +FP)
        
# precision
    if (TP+FP) == 0 :
        precision = 0
    else :
        precision = TP/(TP+FP)

# recall
    recall = sensitivity

# f1_score
    if (precision+recall) == 0 :
        f1_score = 0
    else :
        f1_score = 2*((precision*recall)/(precision+recall))  
    
    return accuracy,sensitivity,specifity,precision,recall,f1_score


In [0]:
def convert(y):
    for i in range(y.shape[0]):
        if y[i]==-1:
            y[i]=1;
        else:
            y[i]=0
    return y

In [0]:
def model_evaluate(ifor,X,y):
    ts = time.time()
    y_pred=ifor.predict(X)
    tf = time.time()
    print('time : ', tf-ts)
    y_pred=convert(y_pred)
    
    accuracy,sensitivity,specifity,precision,recall,f1_score = acc_sens_spec(y,y_pred)
    print('accuracy : ',accuracy)
    print('sensitivity : ',sensitivity)
    print('specifity : ', specifity)
    print('precision : ',precision)
    print('recall : ',recall)
    print('f1 score : ',f1_score)
    print('Confusion matrix')
    TP,FP,FN,TN = confussion_matrik(y,y_pred)
    print('TP = ',TP,'FP = ',FP,'FN = ',FN,'TN = ',TN)
    df_cm = pd.DataFrame([[TN,FP],[FN,TP]],
                  ['True Normal','True Fraud'],['Pred Normal','Pred Fraud'])
    plt.figure(figsize = (8,4))
    sb.set(font_scale=1.4)#for label size
    sb.heatmap(df_cm, annot=True,annot_kws={"size": 16},fmt='g')# font size

In [0]:
def iForest(X, ntree, sample, th):
    ifor = IsolationForest(n_estimators=ntree, max_samples=sample, contamination=th, random_state=1)
#     ifor = IsolationForest(n_estimators=100, max_samples='auto', contamination=0.0039, max_features=1.0, random_state=42, verbose=0)
    ifor.fit(X)
    return ifor

In [0]:
def eval_model(ifor,X,y):
#     ts = time.time()
    y_pred=ifor.predict(X)
#     tf = time.time()
    y_pred=convert(y_pred)
    accuracy,sensitivity,specifity,precision,recall,f1_score = acc_sens_spec(y,y_pred)
    return accuracy,sensitivity,specifity,precision,recall,f1_score

In [0]:
train=pd.read_csv("drive/My Drive/Colab Notebooks/ULB/ULB_preprocessing_train.csv",sep=';')
test = pd.read_csv("drive/My Drive/Colab Notebooks/ULB/hasil_CM_ULB.csv",sep=';')

In [12]:
train.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,52669.0,-1.044875,-0.699877,2.383771,0.462870,-0.523350,-0.484606,-0.782931,0.160622,-1.059381,0.666156,-0.591895,0.378854,2.008465,-0.634986,1.418612,-1.196899,-0.341566,2.081873,-0.795839,0.267347,0.102301,0.625710,0.285710,0.390592,-0.443462,-0.238541,0.448415,0.273184,90.00,0
1,66981.0,-0.105756,1.055263,0.219036,0.972113,0.544316,-1.205159,0.996226,-0.342528,-0.555430,0.305929,-0.402451,0.358783,0.727030,0.308077,0.605273,-0.596082,-0.201596,-0.212436,0.397815,0.061089,0.149714,0.680987,0.163366,0.412736,-0.658454,-0.424248,0.326754,0.220772,2.69,0
2,157487.0,2.273034,-0.620167,-1.579732,-1.038778,-0.195400,-1.021584,-0.241990,-0.453477,-0.649705,0.849121,-1.056267,-0.269228,1.409078,-0.228375,0.256648,1.030782,-0.164283,-1.212026,0.675735,0.059748,0.400569,1.235467,-0.112693,-0.563243,0.332715,0.105684,-0.032988,-0.070058,15.00,0
3,76343.0,-0.868089,-0.127889,1.473924,-1.124669,-1.904013,1.163006,-0.283742,0.746531,-0.736264,-0.175059,0.401805,-0.213943,-0.774657,-0.389873,-1.238179,0.851705,0.798677,-1.065141,1.117206,-0.052033,0.176907,0.469763,0.029259,-0.303608,-0.126885,-0.257276,-0.073542,-0.063614,189.90,0
4,165822.0,-1.240829,-0.395491,1.958589,-0.047792,0.597116,-0.861073,-0.174849,0.156320,0.198213,-0.808267,-0.964026,-0.429859,-0.809422,0.190108,1.242464,-0.312181,0.034655,0.026180,1.173892,0.349427,-0.141820,-0.729206,0.066061,-0.130264,0.157839,-0.011302,0.015111,0.093282,39.95,0


In [13]:
test.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class_aktual,Class_prediksi,Conf Matrik
0,72828.0,-1.142801,1.409180,-0.646741,-1.254767,2.420123,3.376525,-0.204954,0.135930,0.019185,0.357833,0.035796,-0.251040,-0.096891,-0.410270,1.123010,0.670276,-0.445253,0.296747,0.215836,0.273384,0.391299,-1.307445,0.115663,0.913466,0.111612,0.092952,0.369318,0.217595,1.79,0,0,TN
1,160654.0,-1.088132,0.744172,0.761392,-0.861099,0.905130,-0.386599,1.162297,-0.066910,-0.211388,-0.639000,-1.398971,-0.640360,-0.939228,0.210836,-0.303828,-0.136994,-0.301032,-0.502500,0.299998,0.040266,-0.319960,-0.900011,-0.310377,0.469027,0.699531,0.390946,0.075354,0.110781,58.44,0,0,TN
2,69783.0,-9.156731,-1.036810,-4.665348,4.390102,-6.271095,2.299906,0.001721,3.595945,-2.478871,-0.943510,0.435353,1.767629,0.743000,3.402883,1.564858,1.770795,1.703527,-0.090776,0.221188,-1.961031,0.012017,-0.756726,-2.000589,-0.027954,-1.011416,-0.178245,-1.274012,-1.466693,492.91,0,0,TN
3,155489.0,-0.077927,-0.021284,0.961083,-0.315242,-0.132506,-0.489234,0.186363,0.127606,0.119211,-0.338046,0.748297,0.565700,-0.445763,0.175456,-0.353326,0.432332,-0.644371,0.240556,0.477859,0.058023,-0.044945,-0.283158,0.444829,0.047220,-1.311732,0.152113,0.074119,0.115249,59.99,0,0,TN
4,57783.0,-1.794857,1.124720,0.179232,-2.427335,0.196062,-0.969808,0.579130,0.515615,0.199792,-1.432433,0.458249,0.815743,-0.504339,1.016147,0.261781,-0.035981,-0.612508,0.405203,0.921359,-0.169165,-0.297559,-1.002809,-0.336454,-0.554990,0.697254,-1.120106,-0.375169,-0.265168,24.70,0,0,TN


In [14]:
X_train = train.drop('Class',axis=1)
X_train.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,52669.0,-1.044875,-0.699877,2.383771,0.462870,-0.523350,-0.484606,-0.782931,0.160622,-1.059381,0.666156,-0.591895,0.378854,2.008465,-0.634986,1.418612,-1.196899,-0.341566,2.081873,-0.795839,0.267347,0.102301,0.625710,0.285710,0.390592,-0.443462,-0.238541,0.448415,0.273184,90.00
1,66981.0,-0.105756,1.055263,0.219036,0.972113,0.544316,-1.205159,0.996226,-0.342528,-0.555430,0.305929,-0.402451,0.358783,0.727030,0.308077,0.605273,-0.596082,-0.201596,-0.212436,0.397815,0.061089,0.149714,0.680987,0.163366,0.412736,-0.658454,-0.424248,0.326754,0.220772,2.69
2,157487.0,2.273034,-0.620167,-1.579732,-1.038778,-0.195400,-1.021584,-0.241990,-0.453477,-0.649705,0.849121,-1.056267,-0.269228,1.409078,-0.228375,0.256648,1.030782,-0.164283,-1.212026,0.675735,0.059748,0.400569,1.235467,-0.112693,-0.563243,0.332715,0.105684,-0.032988,-0.070058,15.00
3,76343.0,-0.868089,-0.127889,1.473924,-1.124669,-1.904013,1.163006,-0.283742,0.746531,-0.736264,-0.175059,0.401805,-0.213943,-0.774657,-0.389873,-1.238179,0.851705,0.798677,-1.065141,1.117206,-0.052033,0.176907,0.469763,0.029259,-0.303608,-0.126885,-0.257276,-0.073542,-0.063614,189.90
4,165822.0,-1.240829,-0.395491,1.958589,-0.047792,0.597116,-0.861073,-0.174849,0.156320,0.198213,-0.808267,-0.964026,-0.429859,-0.809422,0.190108,1.242464,-0.312181,0.034655,0.026180,1.173892,0.349427,-0.141820,-0.729206,0.066061,-0.130264,0.157839,-0.011302,0.015111,0.093282,39.95


In [17]:
y_train = train['Class'].copy()
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: Class, dtype: int64

In [0]:
test_new = test[(test['Conf Matrik']=='TN') | (test['Conf Matrik']=='FN')]

In [25]:
test_new.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class_aktual,Class_prediksi,Conf Matrik
0,72828.0,-1.142801,1.409180,-0.646741,-1.254767,2.420123,3.376525,-0.204954,0.135930,0.019185,0.357833,0.035796,-0.251040,-0.096891,-0.410270,1.123010,0.670276,-0.445253,0.296747,0.215836,0.273384,0.391299,-1.307445,0.115663,0.913466,0.111612,0.092952,0.369318,0.217595,1.79,0,0,TN
1,160654.0,-1.088132,0.744172,0.761392,-0.861099,0.905130,-0.386599,1.162297,-0.066910,-0.211388,-0.639000,-1.398971,-0.640360,-0.939228,0.210836,-0.303828,-0.136994,-0.301032,-0.502500,0.299998,0.040266,-0.319960,-0.900011,-0.310377,0.469027,0.699531,0.390946,0.075354,0.110781,58.44,0,0,TN
2,69783.0,-9.156731,-1.036810,-4.665348,4.390102,-6.271095,2.299906,0.001721,3.595945,-2.478871,-0.943510,0.435353,1.767629,0.743000,3.402883,1.564858,1.770795,1.703527,-0.090776,0.221188,-1.961031,0.012017,-0.756726,-2.000589,-0.027954,-1.011416,-0.178245,-1.274012,-1.466693,492.91,0,0,TN
3,155489.0,-0.077927,-0.021284,0.961083,-0.315242,-0.132506,-0.489234,0.186363,0.127606,0.119211,-0.338046,0.748297,0.565700,-0.445763,0.175456,-0.353326,0.432332,-0.644371,0.240556,0.477859,0.058023,-0.044945,-0.283158,0.444829,0.047220,-1.311732,0.152113,0.074119,0.115249,59.99,0,0,TN
4,57783.0,-1.794857,1.124720,0.179232,-2.427335,0.196062,-0.969808,0.579130,0.515615,0.199792,-1.432433,0.458249,0.815743,-0.504339,1.016147,0.261781,-0.035981,-0.612508,0.405203,0.921359,-0.169165,-0.297559,-1.002809,-0.336454,-0.554990,0.697254,-1.120106,-0.375169,-0.265168,24.70,0,0,TN


In [0]:
test_act_class = test_new['Class_aktual'].copy()
test_pred_class = test_new['Class_prediksi'].copy()
conf_label = test_new['Conf Matrik'].copy()

In [28]:
test_act_class.head()

0    0
1    0
2    0
3    0
4    0
Name: Class_aktual, dtype: int64

In [0]:
X_test = test_new.drop(['Class_aktual','Class_prediksi','Conf Matrik'], axis=1)

In [30]:
X_test.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,72828.0,-1.142801,1.409180,-0.646741,-1.254767,2.420123,3.376525,-0.204954,0.135930,0.019185,0.357833,0.035796,-0.251040,-0.096891,-0.410270,1.123010,0.670276,-0.445253,0.296747,0.215836,0.273384,0.391299,-1.307445,0.115663,0.913466,0.111612,0.092952,0.369318,0.217595,1.79
1,160654.0,-1.088132,0.744172,0.761392,-0.861099,0.905130,-0.386599,1.162297,-0.066910,-0.211388,-0.639000,-1.398971,-0.640360,-0.939228,0.210836,-0.303828,-0.136994,-0.301032,-0.502500,0.299998,0.040266,-0.319960,-0.900011,-0.310377,0.469027,0.699531,0.390946,0.075354,0.110781,58.44
2,69783.0,-9.156731,-1.036810,-4.665348,4.390102,-6.271095,2.299906,0.001721,3.595945,-2.478871,-0.943510,0.435353,1.767629,0.743000,3.402883,1.564858,1.770795,1.703527,-0.090776,0.221188,-1.961031,0.012017,-0.756726,-2.000589,-0.027954,-1.011416,-0.178245,-1.274012,-1.466693,492.91
3,155489.0,-0.077927,-0.021284,0.961083,-0.315242,-0.132506,-0.489234,0.186363,0.127606,0.119211,-0.338046,0.748297,0.565700,-0.445763,0.175456,-0.353326,0.432332,-0.644371,0.240556,0.477859,0.058023,-0.044945,-0.283158,0.444829,0.047220,-1.311732,0.152113,0.074119,0.115249,59.99
4,57783.0,-1.794857,1.124720,0.179232,-2.427335,0.196062,-0.969808,0.579130,0.515615,0.199792,-1.432433,0.458249,0.815743,-0.504339,1.016147,0.261781,-0.035981,-0.612508,0.405203,0.921359,-0.169165,-0.297559,-1.002809,-0.336454,-0.554990,0.697254,-1.120106,-0.375169,-0.265168,24.70


In [37]:
th = len(y_train[y_train==1])/len(y_train)
th

0.0017254870488152324

In [38]:
ntree = 100
sub_sample = 256
ifor = iForest(X_train,ntree,sub_sample,th)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)


In [39]:
y_pred = ifor.predict(X_test)
y_pred = convert(y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


In [43]:
hasil_pred = test_new.drop(['Class_prediksi','Conf Matrik'], axis=1)
hasil_pred.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class_aktual
0,72828.0,-1.142801,1.409180,-0.646741,-1.254767,2.420123,3.376525,-0.204954,0.135930,0.019185,0.357833,0.035796,-0.251040,-0.096891,-0.410270,1.123010,0.670276,-0.445253,0.296747,0.215836,0.273384,0.391299,-1.307445,0.115663,0.913466,0.111612,0.092952,0.369318,0.217595,1.79,0
1,160654.0,-1.088132,0.744172,0.761392,-0.861099,0.905130,-0.386599,1.162297,-0.066910,-0.211388,-0.639000,-1.398971,-0.640360,-0.939228,0.210836,-0.303828,-0.136994,-0.301032,-0.502500,0.299998,0.040266,-0.319960,-0.900011,-0.310377,0.469027,0.699531,0.390946,0.075354,0.110781,58.44,0
2,69783.0,-9.156731,-1.036810,-4.665348,4.390102,-6.271095,2.299906,0.001721,3.595945,-2.478871,-0.943510,0.435353,1.767629,0.743000,3.402883,1.564858,1.770795,1.703527,-0.090776,0.221188,-1.961031,0.012017,-0.756726,-2.000589,-0.027954,-1.011416,-0.178245,-1.274012,-1.466693,492.91,0
3,155489.0,-0.077927,-0.021284,0.961083,-0.315242,-0.132506,-0.489234,0.186363,0.127606,0.119211,-0.338046,0.748297,0.565700,-0.445763,0.175456,-0.353326,0.432332,-0.644371,0.240556,0.477859,0.058023,-0.044945,-0.283158,0.444829,0.047220,-1.311732,0.152113,0.074119,0.115249,59.99,0
4,57783.0,-1.794857,1.124720,0.179232,-2.427335,0.196062,-0.969808,0.579130,0.515615,0.199792,-1.432433,0.458249,0.815743,-0.504339,1.016147,0.261781,-0.035981,-0.612508,0.405203,0.921359,-0.169165,-0.297559,-1.002809,-0.336454,-0.554990,0.697254,-1.120106,-0.375169,-0.265168,24.70,0


In [0]:
hasil_pred['class_pred'] = y_pred.copy()

In [47]:
hasil_pred.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class_aktual,class_pred
85438,49100.0,1.340946,-0.631690,0.995912,-0.626719,-1.425007,-0.693143,-0.924308,-0.133052,-0.571739,0.566506,-0.129087,-0.030996,1.454154,-0.671208,0.992356,1.429435,-0.010235,-0.987952,0.211403,0.200410,0.405073,1.139645,-0.132189,0.467941,0.459819,-0.055119,0.041539,0.031471,29.95,0,0
85439,162204.0,1.847244,-1.323962,-1.346308,-0.362746,1.104000,4.301386,-1.682068,1.131621,-0.042523,0.928609,-0.448544,0.117496,0.346504,-0.170155,0.914946,-1.000076,-0.602067,1.498562,-1.587012,-0.384522,-0.252816,-0.412175,0.316913,0.619821,-0.444322,-0.739934,0.106879,-0.005560,98.53,0,0
85440,22290.0,1.165435,-0.435033,0.879506,-0.748096,-0.811093,0.190287,-0.891675,0.234495,2.982352,-1.307321,2.002440,-1.514934,0.773231,1.648899,0.197178,-0.433989,0.540630,0.712653,0.409482,-0.206963,-0.107508,0.124804,-0.047611,-0.366065,0.387567,-0.691117,0.071998,0.012176,11.85,0,0
85441,123584.0,2.269104,-1.508913,-0.619983,-1.451776,-1.660494,-0.826669,-1.339567,-0.120379,-0.836715,1.583135,-1.251502,-1.441950,-0.805116,-0.290047,0.119110,-0.421357,0.547580,0.096631,-0.258111,-0.520426,-0.155977,0.028391,0.271831,-0.138301,-0.333039,-0.164670,0.021158,-0.048054,17.40,0,0
85442,59584.0,1.237450,-0.139655,0.569472,0.352674,-0.540186,0.021182,-0.579603,0.234155,0.447212,0.138741,0.339385,-0.309204,-1.547314,0.557821,0.866127,1.095395,-0.991991,0.852702,0.288400,-0.162893,-0.109313,-0.429958,-0.001368,-0.539069,0.180025,0.253563,-0.022985,0.004134,6.42,0,0


In [0]:
aktual = np.array(hasil_pred['Class_aktual'])
predik = np.array(hasil_pred['class_pred'])

In [0]:
conf = []
for i,val in enumerate(aktual):
    if val == 0:
        if val == predik[i]:
            conf.append('TN')
        else:
            conf.append('FP')
    if val == 1:
        if val == predik[i]:
            conf.append('TP')
        else:
            conf.append('FN')

In [52]:
cm = pd.DataFrame(conf)
cm

,0
0,TN
1,TN
2,TN
3,TN
4,TN
5,TN
6,TN
7,TN
8,TN
9,TN


In [0]:
hasil_pred['conf_matrix'] = conf.copy()

In [54]:
hasil_pred

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class_aktual,class_pred,conf_matrix
0,72828.0,-1.142801,1.409180,-0.646741,-1.254767,2.420123,3.376525,-0.204954,0.135930,0.019185,0.357833,0.035796,-0.251040,-0.096891,-0.410270,1.123010,0.670276,-0.445253,0.296747,0.215836,0.273384,0.391299,-1.307445,0.115663,0.913466,0.111612,0.092952,0.369318,0.217595,1.79,0,0,TN
1,160654.0,-1.088132,0.744172,0.761392,-0.861099,0.905130,-0.386599,1.162297,-0.066910,-0.211388,-0.639000,-1.398971,-0.640360,-0.939228,0.210836,-0.303828,-0.136994,-0.301032,-0.502500,0.299998,0.040266,-0.319960,-0.900011,-0.310377,0.469027,0.699531,0.390946,0.075354,0.110781,58.44,0,0,TN
2,69783.0,-9.156731,-1.036810,-4.665348,4.390102,-6.271095,2.299906,0.001721,3.595945,-2.478871,-0.943510,0.435353,1.767629,0.743000,3.402883,1.564858,1.770795,1.703527,-0.090776,0.221188,-1.961031,0.012017,-0.756726,-2.000589,-0.027954,-1.011416,-0.178245,-1.274012,-1.466693,492.91,0,0,TN
3,155489.0,-0.077927,-0.021284,0.961083,-0.315242,-0.132506,-0.489234,0.186363,0.127606,0.119211,-0.338046,0.748297,0.565700,-0.445763,0.175456,-0.353326,0.432332,-0.644371,0.240556,0.477859,0.058023,-0.044945,-0.283158,0.444829,0.047220,-1.311732,0.152113,0.074119,0.115249,59.99,0,0,TN
4,57783.0,-1.794857,1.124720,0.179232,-2.427335,0.196062,-0.969808,0.579130,0.515615,0.199792,-1.432433,0.458249,0.815743,-0.504339,1.016147,0.261781,-0.035981,-0.612508,0.405203,0.921359,-0.169165,-0.297559,-1.002809,-0.336454,-0.554990,0.697254,-1.120106,-0.375169,-0.265168,24.70,0,0,TN
5,65751.0,-0.570796,1.029867,1.249386,0.085165,0.098355,-1.053153,0.635956,-0.093619,-0.284906,-0.385168,-0.237025,-0.183691,-0.077504,-0.425600,0.931915,0.382516,0.011433,-0.117860,-0.077136,0.076230,-0.253906,-0.700053,0.087072,0.326222,-0.155161,0.063436,0.116545,0.094684,1.29,0,0,TN
6,40205.0,-1.647960,0.260289,1.730752,-1.557240,-3.611718,0.980438,3.250190,-0.517123,0.660650,-1.627519,-0.397166,0.434999,0.191594,-1.096162,-1.391661,0.481457,-0.414749,-0.954335,-1.184921,-0.471741,-0.361162,-0.262519,-0.043001,1.049107,0.165816,0.877548,-0.379331,-0.685064,639.15,0,0,TN
7,82024.0,1.089727,0.521074,0.771438,2.314017,0.043398,0.262817,-0.032053,0.168513,-1.039131,0.785976,1.479090,0.864623,0.200544,0.461486,0.083261,0.667632,-0.636027,-0.455897,-0.993486,-0.146768,-0.163925,-0.550664,0.154400,-0.033793,0.152300,-0.254647,0.006738,0.016445,8.08,0,0,TN
8,74270.0,0.939086,-0.211592,1.307988,1.203654,-0.305501,1.650783,-0.861309,0.647216,0.797790,-0.404835,0.521674,1.153710,0.256185,-0.289467,1.204399,-1.300414,1.054990,-2.157197,-1.888903,-0.254828,0.084187,0.611704,0.156915,-0.628640,0.067486,-0.301707,0.146331,0.027902,11.50,0,0,TN
9,119801.0,-1.038696,-0.150409,1.435076,-0.662997,0.789452,0.871091,-0.161542,0.599650,0.530153,-0.750474,-0.739287,-0.235665,-0.802391,-0.093767,0.873987,-0.661120,0.380816,-1.124160,-1.451942,0.098594,0.342579,1.120709,-0.044787,-1.081869,-0.284452,0.737247,0.360802,0.193674,40.20,0,0,TN


In [0]:
nama_file = 'drive/My Drive/Colab Notebooks/ULB/hasil_cm_ULB.csv'
hasil_pred.to_csv(nama_file, sep=',',index=False)

In [57]:
len(test)

85443

In [61]:
hasil_pred['conf_matrix'].value_counts()

TN    85185
FP       99
FN       26
TP        1
Name: conf_matrix, dtype: int64

In [63]:
test_new['Conf Matrik'].value_counts()

TN    85284
FN       27
Name: Conf Matrik, dtype: int64